**Домашнє завдання**

В домашньому завданні до даного модулю ви потренуєтесь робити тестове завдання для влаштування на роботу. За даними акселерометра з мобільного телефону потрібно класифікувати, якою діяльністю займається людина: йде, стоїть, біжить чи йде по сходах. Знайти датасет ви можете за посиланням.

Використайте алгоритми SVM та випадковий ліс з бібліотеки scikit-learn. Як характеристики можете брати показники з акселерометра, проте щоб покращити результати роботи алгоритмів, спочатку можна підготувати наш датасет і розрахувати часові ознаки (time domain features). Більше ці характеристики описані в даній статобою.

Порівняйте результати роботи обох алгоритмів на різних фічах та різні моделі між собою. Використайте метод classification report для порівняння.

Порівняння моделей на основі однієї метрики(такої як Accuracy)- не приймається. Дз повинно бути виконано у Jupyter Nootebook,(або Google Colab) і задеплоїне на Гітхаб у вигляді файлу .ipynb.

In [1]:
import numpy as np
import pandas as pd
from scipy.stats import entropy
from pathlib import Path
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, ParameterGrid
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import accuracy_score, f1_score, classification_report

Будуємо датасет з усіх файлів (може зайняти кілька хвилин):

In [3]:
path = Path("./data")
df = pd.DataFrame()
for el in path.iterdir():
    if el.is_dir():
        for elem in el.iterdir():
            if elem.is_file():
                tmp = pd.read_csv(elem)
                splited_name = elem.stem.rsplit("-")
                tmp.insert(0,"time", int(splited_name[1]))
                tmp.insert(0,"action", el.name)
                df = pd.concat([df, tmp], axis=0, ignore_index=True)
df = df.sort_values(by=["action", "time"], ignore_index=True)
df

,action,time,accelerometer_X,accelerometer_Y,accelerometer_Z
0,idle,1,1.000776,4.616021,8.576031
1,idle,1,0.718261,4.209007,8.446744
2,idle,1,-0.909797,-0.282516,9.203311
3,idle,1,5.099650,0.148441,8.418014
4,idle,1,1.762132,-0.162806,9.251195
...,...,...,...,...,...
193855,walking,1850,4.529830,-12.488156,-0.378284
193856,walking,1850,6.871359,-11.037270,4.620809
193857,walking,1850,7.259220,-6.244078,1.355118
193858,walking,1850,6.143522,0.550666,11.391612


Виконуємо обчислення та будуємо новий датасет з time domain features на основі даних попереднього. Фактично, це відомі (одні більше, інші менше) функції мінімуму, максимуму, медіани, мат. очікування, стандартного відхилення, коефіцієнту асиметрії, двійкової ентропії та міжквартильного розмаху вибірки, розраховані на множині 30-семплових наборів значень прискорення для кожної 3D-осі акселерометра окремо, що відповідають окремим файлам отриманих даних.

In [4]:
df_stats = df.groupby(["action", "time"]).agg(acc_X_min = ('accelerometer_X', 'min'),
                                   acc_X_max = ('accelerometer_X', 'max'),
                                   acc_X_median = ('accelerometer_X', 'median'),
                                   acc_X_mean = ('accelerometer_X', 'mean'),
                                   acc_X_std = ('accelerometer_X', 'std'),
                                   acc_X_skew = ('accelerometer_X', 'skew'),
                                   acc_X_entropy = ('accelerometer_X', lambda x : entropy(x.value_counts(), base=2)),
                                   acc_X_IQR = ('accelerometer_X', lambda x : x.quantile(0.75) - x.quantile(0.25)),
                                   acc_Y_min = ('accelerometer_Y', 'min'),
                                   acc_Y_max = ('accelerometer_Y', 'max'),
                                   acc_Y_median = ('accelerometer_Y', 'median'),
                                   acc_Y_mean = ('accelerometer_Y', 'mean'),
                                   acc_Y_std = ('accelerometer_Y', 'std'),
                                   acc_Y_skew = ('accelerometer_Y', 'skew'),
                                   acc_Y_entropy = ('accelerometer_Y', lambda x : entropy(x.value_counts(), base=2)),
                                   acc_Y_IQR = ('accelerometer_Y', lambda x : x.quantile(0.75) - x.quantile(0.25)),
                                   acc_Z_min = ('accelerometer_Z', 'min'),
                                   acc_Z_max = ('accelerometer_Z', 'max'),
                                   acc_Z_median = ('accelerometer_Z', 'median'),
                                   acc_Z_mean = ('accelerometer_Z', 'mean'),
                                   acc_Z_std = ('accelerometer_Z', 'std'),
                                   acc_Z_skew = ('accelerometer_Z', 'skew'),
                                   acc_Z_entropy = ('accelerometer_Z', lambda x : entropy(x.value_counts(), base=2)),
                                   acc_Z_IQR = ('accelerometer_Z', lambda x : x.quantile(0.75) - x.quantile(0.25)),
                                  ).reset_index()

Розбиваємо вибірку на окремі вибірки з ознаками та цільовою змінною відповідно:

In [5]:
y = df_stats["action"]
X = df_stats.drop(columns=["action"])

Виділяємо окремо навчальні вибірки та тестові:

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [7]:
base_features = [
    "acc_X_min",
    "acc_X_max",
    "acc_X_median",
    "acc_X_mean",
    "acc_Y_min",
    "acc_Y_max",
    "acc_Y_median",
    "acc_Y_mean",
    "acc_Z_min",
    "acc_Z_max",
    "acc_Z_median",
    "acc_Z_mean",
]
additional_features = {
    "no additional features": [],
    "std": ["acc_X_std", "acc_Y_std", "acc_Z_std"],
    "skew": ["acc_X_skew", "acc_Y_skew", "acc_Z_skew"],
    "entropy": ["acc_X_entropy", "acc_Y_entropy", "acc_Z_entropy"],
    "IQR": ["acc_X_IQR", "acc_Y_IQR", "acc_Z_IQR"],
    "all features": [
        "acc_X_std",
        "acc_Y_std",
        "acc_Z_std",
        "acc_X_skew",
        "acc_Y_skew",
        "acc_Z_skew",
        "acc_X_entropy",
        "acc_Y_entropy",
        "acc_Z_entropy",
        "acc_X_IQR",
        "acc_Y_IQR",
        "acc_Z_IQR",
    ],
}

Будуємо SVM (Support Vector Machine) класифікатор та сітку параметрів для навчання:

In [9]:
svm_param_grid = [
    {"estimator__kernel": ["linear"], "estimator__C": [0.1, 1, 10], "estimator__tol": [0.001, 0.0001]},
    {"estimator__kernel": ["rbf"], "estimator__C": [0.1, 1, 10], "estimator__gamma": ["scale", "auto"], "estimator__tol": [0.001, 0.0001]},
    {"estimator__kernel": ["sigmoid"], "estimator__C": [0.1, 1, 10], "estimator__gamma": ["scale", "auto"], "estimator__tol": [0.001, 0.0001]},
    {"estimator__kernel": ["poly"], "estimator__C": [0.1, 1, 10], "estimator__degree": [2, 3, 4], "estimator__gamma": ["scale", "auto"], "estimator__tol": [0.001, 0.0001]},
]

svm_classifiers = {}

Проводимо навчання SVM класифікатора з різними параметрами та з використанням різних наборів ознак (може зайняти кілька десятків хвилин):

In [15]:
for key in additional_features.keys():
    print(f"Training SVM classifier with features: {key}")
    
    svm_classifier = OneVsRestClassifier(
        SVC(probability=True, random_state=42), n_jobs=-1
    )
    
    grid_search = GridSearchCV(
        svm_classifier,
        param_grid=svm_param_grid,
        scoring=["accuracy", "f1_macro"],
        refit="f1_macro",
        cv=3,
        verbose=1,
    )

    selected_features = base_features + additional_features[key]
    grid_search.fit(X_train[selected_features], y_train)
    
    y_pred = grid_search.predict(X_test[selected_features])

    print(f"\nSVM classifier with {key}:")
    print(f"Accuracy: {accuracy_score(y_test, y_pred):.5f}")
    print(f"f1_macro: {f1_score(y_test, y_pred, average='macro'):.5f}")
    print(classification_report(y_test, y_pred))

    svm_classifiers[key] = grid_search

Training SVM classifier with features: no additional features
Fitting 3 folds for each of 66 candidates, totalling 198 fits

SVM classifier with no additional features:
Accuracy: 0.99149
f1_macro: 0.95398
              precision    recall  f1-score   support

        idle       1.00      1.00      1.00       208
     running       1.00      1.00      1.00       682
      stairs       0.84      0.82      0.83        33
     walking       0.98      0.99      0.99       370

    accuracy                           0.99      1293
   macro avg       0.96      0.95      0.95      1293
weighted avg       0.99      0.99      0.99      1293

Training SVM classifier with features: std
Fitting 3 folds for each of 66 candidates, totalling 198 fits

SVM classifier with std:
Accuracy: 0.98995
f1_macro: 0.98580
              precision    recall  f1-score   support

        idle       1.00      1.00      1.00       208
     running       0.98      1.00      0.99       682
      stairs       1.00      0

Будуємо RF (Random Forest) класифікатор та сітку параметрів для навчання:

In [19]:
rf_param_dist = {
    "n_estimators": [10, 50, 100, 200],
    "max_depth": [None, 10, 20, 30],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4],
}

rf_classifier = RandomForestClassifier(random_state=42, n_jobs=8)

rf_classifiers = {}

Проводимо навчання RF класифікатора з різними параметрами та з використанням різних наборів ознак (може зайняти кілька десятків хвилин):

In [21]:
for key in additional_features.keys():
    print(f"\nTraining RF classifier with features: {key}")

    selected_features = base_features + additional_features[key]

    grid_search = GridSearchCV(
        RandomForestClassifier(random_state=42, n_jobs=-1),
        param_grid=rf_param_dist,
        scoring=["accuracy", "f1_macro"],
        refit="f1_macro",
        cv=3,
        verbose=1,
    )

    grid_search.fit(X_train[selected_features], y_train)
    y_pred = grid_search.predict(X_test[selected_features])

    print(f"\nRF classifier with {key}:")
    print(f"Accuracy: {accuracy_score(y_test, y_pred):.5f}")
    print(f"f1_macro: {f1_score(y_test, y_pred, average='macro'):.5f}")
    print(classification_report(y_test, y_pred))

    rf_classifiers[key] = grid_search


Training RF classifier with features: no additional features
Fitting 3 folds for each of 144 candidates, totalling 432 fits

RF classifier with no additional features:
Accuracy: 1.00000
f1_macro: 1.00000
              precision    recall  f1-score   support

        idle       1.00      1.00      1.00       208
     running       1.00      1.00      1.00       682
      stairs       1.00      1.00      1.00        33
     walking       1.00      1.00      1.00       370

    accuracy                           1.00      1293
   macro avg       1.00      1.00      1.00      1293
weighted avg       1.00      1.00      1.00      1293


Training RF classifier with features: std
Fitting 3 folds for each of 144 candidates, totalling 432 fits

RF classifier with std:
Accuracy: 0.99923
f1_macro: 0.99582
              precision    recall  f1-score   support

        idle       1.00      1.00      1.00       208
     running       1.00      1.00      1.00       682
      stairs       1.00      0

Подивимося на параметри найкращих отриманих Random Forest класифікаторів:

In [23]:
rf_classifiers["no additional features"].best_params_

{'max_depth': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 10}

In [25]:
rf_classifiers["entropy"].best_params_

{'max_depth': None,
 'min_samples_leaf': 1,
 'min_samples_split': 5,
 'n_estimators': 50}

In [27]:
for key in rf_classifiers:
    print(f"{key}: {rf_classifiers[key].best_params_}")

no additional features: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 10}
std: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 50}
skew: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 50}
entropy: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 50}
IQR: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 50}
all features: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}


**Висновок**

За результатами навчання моделей, що виконують класифікацію методом опорних векторів (SVM), було отримано доволі очікувані результати — моделі добре справляються із завданням класифікації. Проте один з класів — stairs (рух по сходах) — розпізнається трохи гірше порівняно з іншими класами: idle (стояння), running (біг) та walking (ходьба).
Найкращу якість показали: модель, навчена на повному наборі ознак, а також модель, що використовує мінімальний набір ознак (min, max, median, mean) плюс міжквартильний розмах (IQR).

Неочікувано сильні результати продемонстрували моделі, побудовані за допомогою алгоритму випадкового лісу (Random Forest). Зокрема, дві з них — модель з мінімальним набором ознак, та модель з мінімальним набором плюс ентропія —
змогли досягти повної точності класифікації, тобто не зробили жодної помилки на тестовій вибірці.

Це підтверджує, що алгоритм випадкового лісу є надзвичайно ефективним для задачі класифікації дій за даними акселерометра, навіть без складної інженерії ознак.